<a href="https://colab.research.google.com/github/hrwatts/C-Projects/blob/main/ludcmp_lusolve_example_cpp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Logistic Regression in C++ #

Logistic regression is a fundamental tool of statistical analysis. The model assumes a random sample X and a corresponding response y ~ wX+b are given.

In [637]:
%%writefile matrix.h
/*
*    matrix data structure, and routines for matrix manipulation.
*/
#if !defined(__MATRIX_H)
#define __MATRIX_H
#include <string>
#include <assert.h>

// \typedef
//! pointer to a function which returns a double and takes as argument a double
typedef double (*V_FCT_PTR)(double);

//! defines a vector of doubles
class Vec {
public:
	
	Vec(): n(0), body(0) { } //!< default no-argument constructor
	//! constructs a vector of length n
	Vec(int size): n(0), body(0) { create(size); }
	Vec(const Vec &src): n(0), body(0) { copy(src); } //!< copy constructor
	~Vec() { delete [] body; body=0; n=0; } //!< destructor
	//! creates (or resets) vector length and allocated space
	void create(int size);
	//! returns the length (number of elements) of the vector
	int length() const { return n; }
	//! prints vector contents to stdout
	void print() const;
	//! set to constant value
	Vec &set(double v) { for (int i=0; i<length(); i++) body[i] = v; return *this; }
	//! subscript operator (non-const object)
	double &operator[](int n) {return body[n]; }
	//! subscript operator (const object)
	const double &operator[](int n) const { return body[n]; }
	Vec &operator=(const Vec &src) { return (this==&src? *this: copy(src)); } //!< dst = src
	//! applys function fct element-by-element
	Vec &apply(V_FCT_PTR fct);
	double max(); //!< returns maximum Vec element
	double min(); //!< returns minimum Vec element
	double norm(); //< returns 2-norm (magnitude) of Vec
	void normalize(); //< convert Vec to unit magnitude
	//! returns  dot product of a and b
	static double dot(const Vec &a, const Vec &b);
	
	// operator overloads
	friend Vec operator+(const Vec &a, const Vec &b);  //!< returns a+b
	friend Vec operator-(const Vec &a, const Vec &b); //!< returns a-b
	friend Vec operator*(double c, const Vec &a);     //!< returns scalar multiplied by Vec
	friend Vec operator*(const Vec &a, double c); //!< returns Vec multiplied by scalar
	friend Vec operator*(const Vec &a, double c); //!< returns Vec divided by scalar

private:
	double *body;	            //!< contains the elements of the vector
	int n;	                    //!< number of elements in the vector
	Vec &copy(const Vec &src);  //!< replace current vector with src
};

std::ostream& operator << (std::ostream& s, const Vec& v);

//! defines a two-dimensional Matrix of doubles
class Matrix {
public:
	//! default (no argument) constructor	
	Matrix(): nrows(0), ncols(0), body(0) {}
	//! constructs a Matrix of specified size
	Matrix(int rows, int cols): nrows(0), ncols(0), body(0) { create(rows,cols);}
	Matrix(const Matrix &src): nrows(0), ncols(0), body(0) { copy(src); } //!< copy	constructor
	~Matrix(); //!< destructor
	//! creates(or resets) Matrix size and allocated space.
	void create(int rows, int cols);
	int cols()const { return ncols; } //!< returns the number of columns
	int rows()const { return nrows; } //!< returns the number of rows
	//! set Matrix elements to v
	Matrix &set(double v);
	Matrix &operator =(const Matrix &mx) { return (this==&mx? *this: copy(mx)); }
	Matrix operator *(const Matrix &mx) const;
	//static Vec mul(const Matrix &mx, const Vec &v);
	//static Vec mul(const Vec &v, const Matrix &mx);
	//! swap rows
	void swaprows(int i, int j);
	//! returns the transpose of the Matrix
	Matrix transpose();
	//! returns a pointer to a Matrix row
	double * operator [](int n) const { return body[n]; }
	//const double * operator [](int n) const { return body[n]; }
	//! prints Matrix (using mprint)
	void print() const;
	// friends
	friend Vec operator *(const Matrix &mx, const Vec &v); //!< matix multipled by Vec
	friend Vec operator *(const Vec &v, const Matrix &mx); //!< Vec multiplied by Matrix
private:
	int nrows; //!< number of rows in the Matrix
	int ncols; //!< number of columns in the Matrix
	double **body; //!< space allocated for the Matrix
	Matrix &copy(const Matrix &mx); //!< replaces Matrix with Matrix mx

};
/*! returns the inner (or dot) product of a vector
 * @param a first vector
 * @param b second vector
 * @returns (a dot b)
 */
inline double Vec::dot(const Vec &a, const Vec &b)
{
	double sum = 0.0;
	int n = a.length();
	assert(b.length()==n);
	for (int i=0; i<n; i++) sum += a[i]*b[i];
	return sum;
}



Matrix identity(int size);
Matrix diagonal(double *v, int size);
int read_matrix(Matrix &mx, std::string &title, FILE *in);
void print_vector(double *v, int n);
void copy_matrix(Matrix &dst, Matrix &src);
double *vector(int length);
int *ivector(int length);
void errmsg(char *text);

#endif


Overwriting matrix.h


In [638]:
%%writefile matrix.cpp
/*
*
*    Utility routines for allocating space for matrices and vectors,
*    printing matrices and vectors, and copying one matrix to another.
*/
#include <stdio.h>
#include <stdlib.h>
#include <assert.h>
#include <math.h>
#include <iostream>
using namespace std;
#include "matrix.h"

void Matrix::create(int rows, int cols)
{
	if (rows==nrows && cols==ncols) return;
	while (nrows) delete [] body[--nrows];
	if (body) delete body;
	nrows = rows;
	ncols = cols;
	body = new double * [nrows];
	assert(body);
	for (int i=0; i<nrows; i++) {
		body[i]= new double[ncols];
		assert(body[i]);
	}
}

/*!
*   Copy Matrix src to the current Matrix
*/
Matrix &Matrix::copy(const Matrix &src)
{
	int i, j;
	double *up, *vp;
	create(src.rows(),src.cols());
	for (i=0; i<nrows; i++) {
		up = src[i];
		vp = body[i];
		for (j=0; j<ncols; j++) *vp++ = *up++;
	}
	return *this;
}

Matrix::~Matrix()
{
	while (nrows) delete [] body[--nrows];
	delete [] body;
}

Matrix &Matrix::set(double v)
{
	int i, j;
	double *p;
	for (i=0; i<nrows; i++) {
		p = body[i];
		for (j=0; j<ncols; j++) *p++ = v;
	}
	return *this;
}

void Matrix::swaprows(int i,int j)
{
	double *p = body[i];
	body[i]=body[j];
	body[j]=p;
}

Matrix Matrix::transpose()
{
	int i, j;
	Matrix out(ncols,nrows);
	for (i=0; i<nrows; i++) {
		for (j=0; j<ncols; j++) {
			out[j][i] = body[i][j];
		}
	}
	return out;
}

//! Matrix multiplication

Matrix Matrix::operator *(const Matrix &mx) const
{
	int i, j, k;
	double sum;
	double *vp;
	assert(ncols == mx.rows()); // check for non-conforming Matrix multiplication
	Matrix out(nrows,mx.cols());
	for (i=0; i<nrows; i++) {
		vp = out[i];
		for (j=0; j<mx.cols(); j++) {
			sum = 0.0;
			for (k=0; k<ncols; k++)
				sum += body[i][k] * mx[k][j];
			*vp++ = sum;
		}
	}
	return out;
}

Vec operator *(const Matrix &mx, const Vec &v)
{
	int i, j;
	int m = mx.rows();
	int n = mx.cols();
	double sum;
	assert(m>0 && n>0);
	assert(n==v.length());
	Vec vout(m);
	for (i=0; i<m; i++) {
		sum = 0.0;
		double *ptr = mx[i];
		for (j=0; j<n; j++) sum += (*ptr++) * v[j];
		vout[i] = sum;
	}
	return vout;
}

Vec operator *(const Vec &v, const Matrix &mx)
{
	int i, j;
	int m = mx.rows();
	int n = mx.cols();
	double sum;
	assert(m>0 && n>0);
	assert(m==v.length());
	Vec vout(n);
	for (j=0; j<n; j++) {
		sum = 0.0;
		for (i=0; i<m; i++) sum += v[i] * mx[i][j];
		vout[j] = sum;
	}
	return vout;
}

//!    create an Identity Matrix
Matrix identity(int size)
{
	int i, j;
	double *vp;
	Matrix out(size,size);
	for (i=0; i<size; i++) {
		vp = out[i];
		for (j=0; j<size; j++) vp[j]=0.0;
		vp[i] = 1.0;
	}
	return out;
}


//! create a diagonal Matrix
/*!
 *  @param v is an array of diagonal elements
 *  @param size is the length of the array
 *  
 */
Matrix diagonal(double *v, int size)
{
	int i, j;
	double *vp;
	Matrix out(size,size);
	for (i=0; i<size; i++) {
		vp = out[i];
		for (j=0; j<size; j++) vp[j]=0.0;
		vp[i] = v[i];
	}
	return out;
}

//! read Matrix from text file
/*! @param mx destination Matrix
 *  @param title c-string comment (remainder of header line)
 *  @param in input file
 *  
*  The first two entries are number of rows and columns.\n
*  The rest of the line is a title.\n
*  The body of the Matrix follows.\n
*/

int read_matrix(Matrix &mx, string &title, FILE *in)
{
	int i, j;
	int nrows, ncols;
	double *v;
	double vin;
	char *cp;
	char buf[256];

	fscanf(in," %d %d",&nrows,&ncols);
	if (feof(in)) return -1;
	fgets(buf,255,in);
	if (feof(in)) return -1;
	cp = buf;
	while (*cp) {
		if (*cp=='\n') *cp = 0;
		else cp++;
	}
	title = buf;
	mx.create(nrows,ncols);
	for (j=0; j<nrows; j++) {
		v  = mx[j];
		for (i=0; i<ncols; i++) {
			fscanf(in," %lf",&vin);
			*v++ = vin;
		}
		if (feof(in)) {
			cout << "\nerror reading %s\n" << title;
			cout << "Matrix has %d rows and %d columns\n" <<	mx.rows() << mx.cols();
			cout << "Unexpected EOF reading row %d" << j+1;
			return -1;
		}
	}
	return 0;
}



//! create memory for elements of vector
void Vec::create(int size)
{
	assert(size);
	if (size == n) return;
	if (body != 0) delete [] body;
	body = new double[size];
	assert(body!=0);
	n = size;
	//memset((void *)body, 0, (size_t)(n*sizeof(double));
}
//! copy constructor
Vec &Vec::copy(const Vec &src)
{
	int size = src.length();
	create(size);
	for (int i=0; i<n; i++) body[i] = src[i];
	return *this;
}

double Vec::max()
{
	double vmax = body[0];
	for (int i=1; i<n; i++) if (body[i]>vmax) vmax = body[i];
	return vmax;
}

double Vec::min()
{
	double vmin = body[0];
	for (int i=1; i<n; i++) if (body[i]<vmin) vmin = body[i];
	return vmin;
}

double Vec::norm()
{
	double sum = 0;0;
	for (int i=0; i<n; i++) sum += body[i]*body[i];
	return sqrt(sum);
}

void Vec::normalize()
{
	double vnorm = norm();
	for (int i=0; i<n; i++) body[i] /= vnorm;
}
	

Vec& Vec::apply(V_FCT_PTR fct)
{
	for (int i=0; i<n; i++) body[i] = (*fct)(body[i]);
	return *this;
}

//! returns a + b
Vec operator +(const Vec &a, const Vec &b)
{
	int n = a.length();
	assert(b.length()==n);
	Vec sum(n);
	for (int i=0; i<n; i++) sum[i] = a[i] + b[i];
	return sum;
}

//! returns a - b
Vec operator -(const Vec &a, const Vec &b)
{
	int n = a.length();
	assert(b.length()==n);
	Vec diff(n);
	for (int i=0; i<n; i++) diff[i] = a[i] - b[i];
	return diff;
}

//! returns scalar multiplied by a vector
Vec operator *(double c, const Vec &a)
{
	int n = a.length();
	Vec vout(n);
	for (int i=0; i<n; i++) vout[i] = c*a[i];
	return vout;
}

//! returns vector mutliplied by a scalar
Vec operator *(const Vec &a, double c)
{
	int n = a.length();
	Vec vout(n);
	for (int i=0; i<n; i++) vout[i] = c*a[i];
	return vout;
}

//! returns vector divided by a scalar
Vec operator /(const Vec &a, double c)
{
	int n = a.length();
	Vec vout(n);
	for (int i=0; i<n; i++) vout[i] = a[i]/c;
	return vout;
}

double *vector(int n)
{
	double *v;
	v = new double[n];
	assert(v);
	return v;
}


int *ivector(int n)
{
	int *iv;
	iv = new int[n];
	assert(iv);
	return iv;
}


void Matrix::print() const
{
	int i, j;
	const double tiny = 1e-13;
	double *vp;
	double v;
	for (j=0; j<nrows; j++) {
		vp = body[j];
		for (i=0; i<ncols; i++) {
			v = *vp++;
			if (fabs(v)<tiny) v = 0.0;
			
			printf("% 10.4g", v);
		}
		printf("\n");
	}
}

void Vec::print() const
{
	double *v = body;
	int size = n;
	while (size--) {
		printf(" %10.4g", *v++);
	}
	printf("\n");
}

//! copy one Matrix to another
/*!
 * @param dst destination Matrix
 * @param src source Matrix
 * if destination is smaller than the source, the source is truncated.\n
 * if destination is larger than the source, the remainder is
 * zero-filled\n
 */

void copy_matrix(Matrix &dst, Matrix &src)
{
	int i, j;
	int nrows, ncols;
	double *s, *t;;
	nrows = (dst.rows()<src.rows()? dst.rows(): src.rows());
	ncols = (dst.cols()<src.cols()? dst.cols(): src.cols());
	for (j=0; j<nrows; j++) {
		s = src[j];
		t = dst[j];
		for (i=0; i<ncols; i++) *t++ = *s++;
	}
	// Append zero-filled rows to destination
	for (j=src.rows(); j<nrows; j++) {
		t = dst[j];
		for (i=0; i<ncols; i++) t[i] = 0.0;
	}
	// Append zero-filled columns to destination
	if (src.cols() < ncols) {
		for (j=0; j<dst.rows(); j++) {
			t = dst[j];
			for (i=src.cols(); i<ncols; i++) t[i] = 0.0;
		}
	}
}

// overloaded insertion operator <<
std::ostream& operator << (std::ostream& s, const Vec& v)
{
	char buf[16];
	int n = v.length();
	if (n > 0)
		{
		//int old_precision = s.precision() ; // get current precision
		s << "[" ;
		for (int i=0; i<n; i++)
		{
		cout << buf << 15 << "%.4g" << v[i];
			//s << setprecision (2)
				//<< setiosflags (ios_base::showpoint|ios_base::fixed)
			s << buf ;
			i!=n-1 ? s << ", " : s << "]" ;
			}
      //s << endl ;
		// reset precision and ios flags
		//s.precision (old_precision) ;
		//std::resetiosflags (ios::showpoint|std::ios::fixed) ;
		}
	return s ;
	}


Overwriting matrix.cpp


In [639]:
%%writefile mean.h
#pragma once
double mean(std::vector<double> const& v);
std::vector<double> podds(std::vector<double> const& b,std::vector<double> const& x);

Overwriting mean.h


In [640]:
%%writefile mean.cpp
#include <vector>
#include <numeric>
#include <iostream>
#include <cmath>

double mean(std::vector<double> const& v){
    if(v.empty()){
        return 0;
    };

    auto const count = static_cast<double>(v.size());
    double bar = 0;
    for(int i = 0; i < count; i++) {
        bar = bar+v[i];
    }
    return bar/count;
    };

std::vector<double> podds(std::vector<double> const& b,std::vector<double> const& x){
    auto const countn = static_cast<double>(x.size());
    std::vector<double> rab = x;
    for(int i=0;i<countn;i++){
      rab[i] = 1/(1+exp(-(b[0] + b[1]*x[i])));
    }
    return rab;
    };

Overwriting mean.cpp


In [641]:
%%writefile ludcmp.h
#pragma once
#include <stdio.h>
#include <stdlib.h>
#include <assert.h>
#include <math.h>
#include "matrix.h"
int ludcmp(Matrix &a, int order[]);
int pivot(Matrix &a, int order[], int jcol);
void solvlu(const Matrix &a, const Vec &b, Vec &x, const int order[]);

Overwriting ludcmp.h


In [642]:
%%writefile ludcmp.cpp
/*
*
*!  Finds solution to set of linear equations A x = b by LU decomposition.
*
*  Chapter 2, Programs 3-5, Fig. 2.8-2.10
*  Gerald/Wheatley, APPLIED NUMERICAL ANALYSIS (fourth edition)
*  Addison-Wesley, 1989
*
*/
#include <stdio.h>
#include <stdlib.h>
#include <assert.h>
#include <math.h>
#include "matrix.h"

int ludcmp(Matrix &a, int order[]);
void solvlu(const Matrix &a, const Vec &b, Vec &x, const int order[]);
static int pivot(Matrix &a, int order[], int jcol);

#define TINY 1e-20
//! finds LU decomposition of Matrix
/*!
*   The function ludcmp computes the lower L and upper U triangular
*   matrices equivalent to the A Matrix, such that L U = A.  These
*   matrices are returned in the space of A, in compact form.
*   The U Matrix has ones on its diagonal.  Partial pivoting is used
*   to give maximum valued elements on the diagonal of L.  The order of
*   the rows after pivoting is returned in the integer vector "order".
*   This should be used to reorder the right-hand-side vectors before
*   solving the system A x = b.
*
*
* \param       a     - n by n Matrix of coefficients
* \param      order - integer vector holding row order after pivoting.
*
*/

int ludcmp(Matrix &a, int order[])
{
	int i, j, k, n, nm1;
	int flag = 1;    /* changes sign with each row interchange */
	double sum, diag;

	n = a.rows();
	assert(a.cols()==n);

	/* establish initial ordering in order vector */
	
	for (i=0; i<n; i++) order[i] = i;

	/* do pivoting for first column and check for singularity */

	if (pivot(a,order,0)) flag = -flag;
	diag = 1.0/a[0][0];
	for (i=1; i<n; i++) a[0][i] *= diag;
	
	/* 
	*  Now complete the computing of L and U elements.
	*  The general plan is to compute a column of L's, then
	*  call pivot to interchange rows, and then compute
	*  a row of U's.
	*/
	
	nm1 = n - 1;
	for (j=1; j<nm1; j++) {
		/* column of L's */
		for (i=j; i<n; i++) {
			sum = 0.0;
			for (k=0; k<j; k++) sum += a[i][k]*a[k][j];
			a[i][j] -= sum;
		}
		/* pivot, and check for singularity */
		if (pivot(a,order,j)) flag = -flag;
		/* row of U's */
		diag = 1.0/a[j][j];
		for (k=j+1; k<n; k++) {
			sum = 0.0;
			for (i=0; i<j; i++) sum += a[j][i]*a[i][k];
			a[j][k] = (a[j][k]-sum)*diag;
		}
	}

	/* still need to get last element in L Matrix */

	sum = 0.0;
	for (k=0; k<nm1; k++) sum += a[nm1][k]*a[k][nm1];
	a[nm1][nm1] -= sum;
	return flag;
}

//!  Find pivot element
/*!
*   The function pivot finds the largest element for a pivot in "jcol"
*   of Matrix "a", performs interchanges of the appropriate
*   rows in "a", and also interchanges the corresponding elements in
*   the order vector.
*
*
*  \param     a      -  n by n Matrix of coefficients
*  \param   order  - integer vector to hold row ordering
*  \param    jcol   - column of "a" being searched for pivot element
*
*/
int pivot(Matrix &a, int order[], int jcol)
{
	int i, ipvt,n;
	double big, anext;
	n = a.rows();

	/*
	*  Find biggest element on or below diagonal.
	*  This will be the pivot row.
	*/

	ipvt = jcol;
	big = fabs(a[ipvt][ipvt]);
	for (i = ipvt+1; i<n; i++) {
		anext = fabs(a[i][jcol]);
		if (anext>big) {
			big = anext;
			ipvt = i;
		}
	}
	assert(fabs(big)>TINY); // otherwise Matrix is singular
	
	/*
	*   Interchange pivot row (ipvt) with current row (jcol).
	*/
	
	if (ipvt==jcol) return 0;
	a.swaprows(jcol,ipvt);
	i = order[jcol];
	order[jcol] = order[ipvt];
	order[ipvt] = i;
	return 1;
}

//!  This function is used to find the solution to a system of equations,
/*!   A x = b, after LU decomposition of A has been found.
*    Within this routine, the elements of b are rearranged in the same way
*    that the rows of a were interchanged, using the order vector.
*    The solution is returned in x.
*
*
*  \param  a     - the LU decomposition of the original coefficient Matrix.
*  \param  b     - the vector of right-hand sides
*  \param  x     - the solution vector
*  \param  order - integer array of row order as arranged during pivoting
*
*/
void solvlu(const Matrix &a, const Vec &b, Vec &x, const int order[])
{
	int i,j,n;
	double sum;
	n = a.rows();

	/* rearrange the elements of the b vector. x is used to hold them. */

	for (i=0; i<n; i++) {
		j = order[i];
		x[i] = b[j];
	}

	/* do forward substitution, replacing x vector. */

	x[0] /= a[0][0];
	for (i=1; i<n; i++) {
		sum = 0.0;
		for (j=0; j<i; j++) sum += a[i][j]*x[j];
		x[i] = (x[i]-sum)/a[i][i];
	}

	/* now get the solution vector, x[n-1] is already done */

	for (i=n-2; i>=0; i--) {
		sum = 0.0;
		for (j=i+1; j<n; j++) sum += a[i][j] * x[j];
		x[i] -= sum;
	}
}

Overwriting ludcmp.cpp


In [644]:
%%writefile lusolve.h
#pragma once
#include <stdio.h>
#include <stdlib.h>
#include <assert.h>
#include <math.h>
#include "matrix.h"
void print_vector(double *v, int n);
void lusolve(Matrix &mx, FILE *in);
int ludcmp(Matrix &a, int order[]);
void solvlu(const Matrix &a, const Vec &b, Vec &x, const int order[]);


Overwriting lusolve.h


In [645]:
%%writefile  lusolve.cpp
/*
*   Solve linear equations by LU decomposition
*/
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include "matrix.h"
#include <string>
using namespace std;

char text[128];
void print_vector(double *v, int n);


void lusolve(Matrix &mx, FILE *in);
int ludcmp(Matrix &a, int order[]);
void solvlu(const Matrix &a, const Vec &b, Vec &x, const int order[]);

void lusolve(Matrix &mx, FILE *in)
{
	int i, j, k, n, flag, index, nrhs;
	int *ipvt;
	double sum;
	Vec b, x, save;
	double det;
	/*
	*   Create matrix a as a copy of input matrix.
	*/
	assert(mx.rows()==mx.cols()); // input must be square matrix
	n = mx.rows();

	Matrix a = mx;
	ipvt = ivector(n);

	flag = ludcmp(a,ipvt);

	/* Calculate determinant */

	det = flag;
	for (i=0; i<n; i++) det *= a[i][i];

	printf("\ndeterminant: %g\n",det);
	printf("pivot vector: ");
	for (i=0; i<n; i++) printf("%3d",ipvt[i]);
	printf("\n");

	/* print LU Matrix */

	Matrix lower = a;
	Matrix upper = a;
	Matrix product(n, n);

	for (i=0; i<n; i++) {
		for (j=i+1; j<n; j++) lower[i][j] = 0.0;
		for (j=0; j<i; j++) upper[i][j] = 0.0;
		upper[i][i] = 1.0;
	}
	printf("\nLower triangular Matrix\n");
	lower.print();
	printf("\nUpper triangular Matrix\n");
	upper.print();

	/* Multiply the lower and upper matrices */

	for (i=0; i<n; i++) {
		for (j=0; j<n; j++) {
			sum = 0.0;
			for (k=0; k<n; k++) sum += lower[i][k] * upper[k][j];
			product[i][j] = sum;
		}
	}
	printf("\n\nProduct of L U\n\n");
	product.print();

	/*
	*   Read in the right-hand side vectors.
	*/
	fscanf(in," %d",&nrhs);
	fgets(text,72,in);
	x.create(n);
	b.create(n);
	index = 0;
	while (nrhs--) {
		for (i=0; i<n; i++) fscanf(in," %lf", &b[i]);
		if (feof(in)) break;
		printf("\nRight-hand-side number %d\n\n",++index);
		b.print();
		save = b;

		if (fabs(det)<1e-10) {
			printf("\nCoefficient Matrix is singular.\n");
			continue;
		}
		solvlu(a,b,x,ipvt);
		printf("\nSolution vector\n\n");
		x.print();
				
		k = 0;
		for (i=0; i<n; i++) {
			sum = 0.0;
			for (j=0; j<n; j++) sum += mx[i][j]*x[j];
			if (fabs(sum-save[i])>1e-8) k++;
		}
		printf("\nsolution %s.\n",(k? "does not check": "checks"));
	}
	free(ipvt);
}

void print_vector(double *v, int n)
{
	while (n--) printf("%10.4f", *v++);
	printf("\n");
}


Overwriting lusolve.cpp


In [646]:
%%writefile main.cpp
#include <vector>
#include <numeric>
#include <iostream>
#include "mean.h"
#include "matrix.h"
#include "ludcmp.h"
#include "lusolve.h"
#include <string>

int main(int argc, char *argv[])
{
	Matrix mx;
	FILE *in;
	char *filename;
  char text2[128];

	if (argc>1) {
		filename = argv[1];
		std::cout << "opening: %s\n"  << filename;
	}
	else {
		std::cout << "This program requires a data file.\n";
		std::cout <<"Enter filename: ";
		std::cin.getline(text2,72);
		if (*text2==0 || *text2=='\n') return -1;
		filename = text2;
	}
	in = std::fopen(filename,"r+");
	if (!in) {
		std::perror(filename);
		return -1;
	}

	std::string title;
	while (read_matrix(mx,title,in)==0) {
		std::cout << "\n%s\n" << title.c_str();
		std::cout << "\nMatrix A\n";
		mx.print();
		lusolve(mx, in);
	}
	return 0;
}

Overwriting main.cpp


In [647]:
%%script bash

g++ main.cpp mean.cpp ludcmp.cpp lusolve.cpp matrix.cpp -std=c++11 -o test 

In [648]:
!./test

This program requires a data file.
Enter filename: eqn5.dat

%s
  Example set of equations used in text, beginning on p. 113
Matrix A
         4        -2         1
        -3        -1         4
         1        -1         3

determinant: -18
pivot vector:   0  1  2

Lower triangular Matrix
         4         0         0
        -3      -2.5         0
         1      -0.5       1.8

Upper triangular Matrix
         1      -0.5      0.25
         0         1      -1.9
         0         0         1


Product of L U

         4        -2         1
        -3        -1         4
         1        -1         3

Right-hand-side number 1

         15          8         13

Solution vector

          2         -2          3

solution checks.

%s
  Example set of equations used in text, beginning on p. 120
Matrix A
         0         2         0         1
         2         2         3         2
         4        -3         0         1
         6         1        -6        -5

determinant: -